# Normalize Datasets

The Dataset Normalizer plugin is used to transform 'pandas-unfriendly' datasets (e.g., Excel files that do not follow a standard tabular structure) into a more suitable format for pandas. It is backed by an LLM that generates Python code to convert the original datasets into new ones.

In `tablegpt-agent`, this plugin is used to better format 'pandas-unfriendly' datasets, making them more understandable for the subsequent steps. This plugin is optional; if used, it serves as the very first step in the [File Reading Workflow](../../explanation/file-reading), easing the difficulty of data analysis in the subsequent workflow.

## Introduction

The `Dataset Normalizer` is a specialized tool designed to tackle challenges that arise when working with irregular and poorly structured datasets. These challenges are especially prevalent in Excel files, which are often used as a flexible but inconsistent way of storing data.

Analyzing Excel data files can pose significant challenges, such as:

- **Irregular Formatting:** Datasets may lack a consistent tabular structure, with varying cell sizes or non-standard layouts.
- **Merged Cells:** Cells spanning multiple rows or columns can disrupt parsing tools.
- **Inconsistent Headers:** Columns may have incomplete, redundant, or nested headers.
- **Hidden Data:** Data may be stored in additional sheets or rely on calculated fields that are not directly accessible.
- **Mixed Data Types:** Columns may contain inconsistent data types, such as numbers mixed with text.
- **Empty or Placeholder Rows:** Extra rows with missing or irrelevant data can complicate data loading and analysis.

> **!!! Note:** When the `tablegpt-agent` enables the `Dataset Normalizer` to format the dataset, the dataset reading process will be noticeably slower. This is because the `Dataset Normalizer` needs to analyze the dataset and generate transformation code, a process that takes considerable time. 
>
> **It is worth noting that the data normalization process can effectively address most common data irregularities. However, for more complex datasets, further optimization may be needed, and the results depend on the specific normalization model used.**

## Quick Start

To enable the `Dataset Normalizer`, ensure you pass it as a parameter when creating the `tablegpt-agent`. You can follow the example below:

In [7]:
from pathlib import Path
from langchain_openai import ChatOpenAI
from pybox import LocalPyBoxManager
from tablegpt.agent import create_tablegpt_graph
from tablegpt import DEFAULT_TABLEGPT_IPYKERNEL_PROFILE_DIR

llm = ChatOpenAI(openai_api_base="YOUR_VLLM_URL", openai_api_key="whatever", model_name="TableGPT2-7B")
normalize_llm = ChatOpenAI(openai_api_base="YOUR_VLLM_URL", openai_api_key="whatever", model_name="YOUR_VLLM_MODEL_NAME")
pybox_manager = LocalPyBoxManager(profile_dir=DEFAULT_TABLEGPT_IPYKERNEL_PROFILE_DIR)

agent = create_tablegpt_graph(
    llm=llm,
    pybox_manager=pybox_manager,
    normalize_llm=normalize_llm,
)

Given an Excel file [产品生产统计表.xlsx](https://github.com/tablegpt/tablegpt-agent/blob/main/examples/datasets/产品生产统计表.xlsx) with merged cells and irregular headers:

<table style="border: 1px solid black; border-collapse: collapse;">
  <thead>
    <tr>
      <th colspan="9" style="text-align: center; font-size: 24px;">产品生产统计表</th>
    </tr>
    <tr>
      <th rowspan="2">生产日期</th>
      <th rowspan="2">制造编号</th>
      <th rowspan="2">产品名称</th>
      <th rowspan="2">预定产量</th>
      <th colspan="2">本日产量</th>
      <th rowspan="2">累计产量</th>
      <th colspan="2">耗费工时</th>
    </tr>
    <tr>
      <th>预计</th>
      <th>实际</th>
      <th>本日</th>
      <th>累计</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2007/8/10</td>
      <td>FK-001</td>
      <td>猕猴桃果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>45000</td>
      <td>83000</td>
      <td>10</td>
      <td>20</td>
    </tr>
    <tr>
      <td>2007/8/11</td>
      <td>FK-002</td>
      <td>西瓜果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>44000</td>
      <td>82000</td>
      <td>9</td>
      <td>18</td>
    </tr>
    <tr>
      <td>2007/8/12</td>
      <td>FK-003</td>
      <td>草莓果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>45000</td>
      <td>83000</td>
      <td>9</td>
      <td>18</td>
    </tr>
    <tr>
      <td>2007/8/13</td>
      <td>FK-004</td>
      <td>蓝莓果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>45000</td>
      <td>83000</td>
      <td>9</td>
      <td>18</td>
    </tr>
  </tbody>
</table>

Add the file for processing in the `additional_kwargs` of HumanMessage:

In [8]:
from typing import TypedDict
from langchain_core.messages import HumanMessage

class Attachment(TypedDict):
    """Contains at least one dictionary with the key filename."""
    filename: str

attachment_msg = HumanMessage(
    content="",
    # Please make sure your iPython kernel can access your filename.
    additional_kwargs={"attachments": [Attachment(filename="产品生产统计表.xlsx")]},
)

Invoke the `tablegpt-agent` to normalize the datasets:

In [9]:
from datetime import date
from tablegpt.agent.file_reading import Stage

# Reading and processing files.
response = await agent.ainvoke(
    input={
        "entry_message": attachment_msg,
        "processing_stage": Stage.UPLOADED,
        "messages": [attachment_msg],
        "parent_id": "some-parent-id1",
        "date": date.today(),
    },
    config={
        # Using checkpointer requires binding thread_id at runtime.
        "configurable": {"thread_id": "some-thread-id"},
    },
)

response["messages"]

[HumanMessage(content='', additional_kwargs={'attachments': [{'filename': '产品生产统计表.xlsx'}]}, response_metadata={}, id='452ef08b-a45f-49ce-8489-4062cd35d57c'),
 AIMessage(content='我已经收到您的数据文件，我需要查看文件内容以对数据集有一个初步的了解。首先我会读取数据到 `df` 变量中，并通过 `df.info` 查看 NaN 情况和数据类型。\n```python\n# Load the data into a DataFrame\ndf = read_df(\'产品生产统计表.xlsx\')\n# Normalize the data\ntry:\n    df = df.copy()\n\n    import pandas as pd\n\n    # Assuming the original data is loaded into a DataFrame named `df`\n\n    # Step 1: Isolate the Table Header\n    # Remove the unnecessary top rows and columns\n    df = df.drop([0, 1])  # Drop the first two rows which contain header information\n    df = df.reset_index(drop=True)  # Reset the index to start from 0\n\n    # Step 2: Store the Result as `final_df`\n    final_df = df\n\n    # Step 3: Rename Columns of final_df\n    # Adjust the column names of final_df to match the desired format\n    final_df.columns = [\'生产日期\', \'制造编号\', \'产品名称\', \'预定产量\', \'本日产量预计\', \'

By formatting the content of the last `ToolMessage`, you can see the normalized data:

<table style="border: 1px solid black; border-collapse: collapse;">
  <thead>
    <tr>
      <th>生产日期</th>
      <th>制造编号</th>
      <th>产品名称</th>
      <th>预定产量</th>
      <th>本日产量预计</th>
      <th>本日产量实际</th>
      <th>累计产量</th>
      <th>本日耗费工时</th>
      <th>累计耗费工时</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2007/8/10</td>
      <td>FK-001</td>
      <td>猕猴桃果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>45000</td>
      <td>83000</td>
      <td>10</td>
      <td>20</td>
    </tr>
    <tr>
      <td>2007/8/11</td>
      <td>FK-002</td>
      <td>西瓜果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>44000</td>
      <td>82000</td>
      <td>9</td>
      <td>18</td>
    </tr>
    <tr>
      <td>2007/8/12</td>
      <td>FK-003</td>
      <td>草莓果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>45000</td>
      <td>83000</td>
      <td>9</td>
      <td>18</td>
    </tr>
    <tr>
      <td>2007/8/13</td>
      <td>FK-004</td>
      <td>蓝莓果肉饮料</td>
      <td>100000</td>
      <td>40000</td>
      <td>45000</td>
      <td>83000</td>
      <td>9</td>
      <td>18</td>
    </tr>
  </tbody>
</table>